In [11]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [12]:
class ExpDataset(Dataset):
    def __init__(self, csv_file):
        self.data = pd.read_csv(csv_file)
        self.features = self.data.iloc[:, :-1].values.astype(np.float32)
        self.labels = self.data.iloc[:, -1].values
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        image = self.features[idx].reshape(28, 28)  # Assuming 28x28 images
        label = self.labels[idx]
        
        # Convert to PyTorch tensors
        image = torch.from_numpy(image).unsqueeze(0)  # Add channel dimension
        label = torch.tensor(label, dtype=torch.long)
        
        return image, label


In [13]:
# Load the datasets
train_data = ExpDataset('train_data.csv')
val_data = ExpDataset('val_data.csv')
test_data = ExpDataset('test_data.csv')

# Create data loaders
batch_size = 64
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size)
test_loader = DataLoader(test_data, batch_size=batch_size)

In [14]:
class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.pool1(x)
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.pool2(x)
        x = x.view(-1, 64 * 7 * 7)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [15]:
num_classes = 16  # Digits 0-9, operators +, -, *, /, and brackets (, )
model = CNN(num_classes)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)



In [20]:
patience = 3  # Number of epochs to wait for improvement
min_delta = 0.001  # Minimum change in validation loss to qualify as an improvement
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model = model.to(device)
num_epochs = 100
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)


best_val_loss = float('inf')
epochs_without_improvement = 0

# Training loop
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    train_correct = 0
    train_total = 0
    
    for batch_idx, (data, target) in enumerate(test_loader):
        data, target = data.to(device), target.to(device)
        
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = output.max(1)
        train_total += target.size(0)
        train_correct += predicted.eq(target).sum().item()
        
        if batch_idx % 100 == 0:
            print(f'Epoch {epoch+1}/{num_epochs}, Batch {batch_idx}/{len(test_loader)}, Loss: {loss.item():.4f}')
    
    train_loss /= len(train_loader)
    train_accuracy = train_correct / train_total
    
    # Validation
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    
    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            val_loss += criterion(output, target).item()
            _, predicted = output.max(1)
            val_total += target.size(0)
            val_correct += predicted.eq(target).sum().item()
    
    val_loss /= len(val_loader)
    val_accuracy = val_correct / val_total
    
    # Step the learning rate scheduler
    scheduler.step()
    
    print(f'Epoch {epoch+1}/{num_epochs}:')
    print(f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}')
    print(f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')
    print(f'Learning Rate: {scheduler.get_last_lr()[0]:.6f}')
    
    # Early stopping check
    if val_loss < best_val_loss - min_delta:
        best_val_loss = val_loss
        epochs_without_improvement = 0
        # Save the best model
        torch.save(model.state_dict(), 'best_model_cnn2.pth')
    else:
        epochs_without_improvement += 1
    
    if epochs_without_improvement >= patience:
        print(f'Early stopping triggered after {epoch+1} epochs')
        break
    
    print('---')

# Load the best model for testing
model.load_state_dict(torch.load('best_model_cnn2.pth'))

Using device: cpu
Epoch 1/100, Batch 0/1044, Loss: 0.2511
Epoch 1/100, Batch 100/1044, Loss: 0.2426
Epoch 1/100, Batch 200/1044, Loss: 0.1674
Epoch 1/100, Batch 300/1044, Loss: 0.0428
Epoch 1/100, Batch 400/1044, Loss: 0.0805
Epoch 1/100, Batch 500/1044, Loss: 0.0507
Epoch 1/100, Batch 600/1044, Loss: 0.1203
Epoch 1/100, Batch 700/1044, Loss: 0.0667
Epoch 1/100, Batch 800/1044, Loss: 0.0251
Epoch 1/100, Batch 900/1044, Loss: 0.2638
Epoch 1/100, Batch 1000/1044, Loss: 0.0663
Epoch 1/100:
Train Loss: 0.0296, Train Accuracy: 0.9718
Val Loss: 0.0713, Val Accuracy: 0.9771
Learning Rate: 0.000010
---
Epoch 2/100, Batch 0/1044, Loss: 0.0827
Epoch 2/100, Batch 100/1044, Loss: 0.0590
Epoch 2/100, Batch 200/1044, Loss: 0.1073
Epoch 2/100, Batch 300/1044, Loss: 0.0965
Epoch 2/100, Batch 400/1044, Loss: 0.0481
Epoch 2/100, Batch 500/1044, Loss: 0.0792
Epoch 2/100, Batch 600/1044, Loss: 0.1936
Epoch 2/100, Batch 700/1044, Loss: 0.0929
Epoch 2/100, Batch 800/1044, Loss: 0.0392
Epoch 2/100, Batch 900

<All keys matched successfully>

In [25]:
# Test the model

model = CNN(num_classes=16)  # Adjust num_classes if needed
model.load_state_dict(torch.load("best_model_cnn2.pth"))
model.eval()
test_loss = 0.0
test_correct = 0
test_total = 0

# Define class mapping
class_mapping = {
    0: '0', 1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6', 7: '7', 8: '8', 9: '9',
    10: '+', 11: '-', 12: '*', 13: '/', 14: '(', 15: ')'
}

# List to store sample predictions
sample_predictions = []

with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        test_loss += criterion(output, target).item()
        _, predicted = output.max(1)
        test_total += target.size(0)
        test_correct += predicted.eq(target).sum().item()
        
        # Store some sample predictions
        for true, pred in zip(target[:5], predicted[:5]):  # Store up to 5 samples per batch
            sample_predictions.append((true.item(), pred.item()))
        
        if len(sample_predictions) >= 20:  # Stop after collecting 20 samples
            break

test_loss /= len(test_loader)
test_accuracy = test_correct / test_total

print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')

# Print sample predictions
print("\nSample Predictions:")
for i, (true, pred) in enumerate(sample_predictions[:20], 1):
    print(f"Sample {i}:")
    print(f"Ground Truth: {class_mapping[true]}")
    print(f"Prediction: {class_mapping[pred]}")
    print(f"Correct: {true == pred}")
    print("---")

Test Loss: 0.0003, Test Accuracy: 0.9727

Sample Predictions:
Sample 1:
Ground Truth: (
Prediction: (
Correct: True
---
Sample 2:
Ground Truth: 4
Prediction: 4
Correct: True
---
Sample 3:
Ground Truth: *
Prediction: *
Correct: True
---
Sample 4:
Ground Truth: 0
Prediction: 0
Correct: True
---
Sample 5:
Ground Truth: 9
Prediction: 9
Correct: True
---
Sample 6:
Ground Truth: *
Prediction: *
Correct: True
---
Sample 7:
Ground Truth: 5
Prediction: 5
Correct: True
---
Sample 8:
Ground Truth: -
Prediction: -
Correct: True
---
Sample 9:
Ground Truth: 3
Prediction: 5
Correct: False
---
Sample 10:
Ground Truth: -
Prediction: -
Correct: True
---
Sample 11:
Ground Truth: 1
Prediction: 1
Correct: True
---
Sample 12:
Ground Truth: 0
Prediction: 0
Correct: True
---
Sample 13:
Ground Truth: /
Prediction: /
Correct: True
---
Sample 14:
Ground Truth: +
Prediction: +
Correct: True
---
Sample 15:
Ground Truth: 6
Prediction: 6
Correct: True
---
Sample 16:
Ground Truth: 8
Prediction: 8
Correct: True
---
Sa

In [18]:
# Test the model
model.eval()
test_loss = 0.0
test_correct = 0
test_total = 0

with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        test_loss += criterion(output, target).item()
        _, predicted = output.max(1)
        test_total += target.size(0)
        test_correct += predicted.eq(target).sum().item()

test_loss /= len(test_loader)
test_accuracy = test_correct / test_total

print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')

Test Loss: 0.0445, Test Accuracy: 0.9846
